In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import openpyxl  # openpyxl 모듈 추가
import re

sns.set_style('darkgrid')
sns.set_palette('Set3')
warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = "NanumBarunGothic"


In [10]:
import mysql.connector

# 데이터베이스 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='music',
    database='MusicDatabase'
)

cursor = conn.cursor()

# 쿼리 실행  기존에 존재하는 데이터는 제외하고  출력
# 각 플랫폼별 최신일자 Top10에 대한 정보 출력 ※idol_Group테이블에는 Top10에 대한 그룹정보 저장 
# 각 플랫폼별 최신일자 Top10에 대한 싱어코드 추출하여 싱어코드를 바탕으로 멜론 사이트에서 그룹정보 크롤링
query = """
    SELECT Singer_Code
    FROM Music
    WHERE Day = (SELECT max(Day) FROM Music)
    AND Ranking < 11
    AND Singer_Code not in (select Group_Code from idol_Group group by Group_Code)
    AND Singer_Code IS NOT NULL
    GROUP BY Singer_Code
    ORDER BY MIN(Ranking) ASC;
"""
cursor.execute(query)

# 결과 출력
Singer_Code_List = cursor.fetchall()
for (Singer_Code) in Singer_Code_List:
    print(Singer_Code)

# 커서와 연결 닫기
cursor.close()
conn.close()



(995173,)
(712452,)
(2399724,)


In [11]:

Singer_Code_List

# 튜플의 첫 번째 요소만 추출
singer_codes = [code[0] for code in Singer_Code_List]

print(singer_codes)


[995173, 712452, 2399724]


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# Chrome WebDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 타임라인 페이지로 이동
for i in singer_codes:
    URL = "https://www.melon.com/artist/timeline.htm?artistId={0}".format(i)
    driver.get(URL)

    # 페이지 로딩 대기
    time.sleep(1)

    detailed_info_link = driver.find_element(By.XPATH, "//li/a[contains(@title, '상세정보')]")
    detailed_info_link.click()

    time.sleep(1)

    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')

    conts = soup.select("#conts")

    for x in conts:
        # 데뷔년도
        debut_Year = x.select_one("#conts > div.section_atistinfo03 > dl > dd:nth-child(2)")
        G_Name = x.select_one("#conts > div.wrap_dtl_atist > div > div.wrap_atist_info > p")
        G_Img = x.select_one("#artistImgArea > img")
        debut_song = x.select_one("#conts > div.section_atistinfo03 > div > div.debutsong_info_area > div > dl > dd > div > a")
        Year_Active = x.select_one("#conts > div.section_atistinfo03 > dl > dd:nth-child(4)")
        Group_type = x.select_one("#conts > div.section_atistinfo03 > dl > dd:nth-child(6)")
        Genre_Text = x.select_one("#conts > div.section_atistinfo03 > dl > dd:nth-child(8)")
        Company = x.select_one("#conts > div.section_atistinfo03 > dl > dd:nth-child(12) > a")
        if debut_Year:
            # .text로 문자열로 변환 
            debut_year_text = debut_Year.text.strip()  # 공백 제거
            G_Name_text = G_Name.text.strip()  # 공백 제거
            G_Name_text = G_Name_text.replace("아티스트명", "").strip()
            img_url = G_Img.get('src')
            if debut_song:
                debut_song = debut_song.get('a')
            else:
                debut_song = "" 

            if Year_Active:
                Year_Active = Year_Active.text.replace("\n", "").replace("\t", "").replace("|", " | ").strip()  
            else:
                Year_Active = ""  #

            if Group_type:
                Group_type = Group_type.text.replace("\n", "").replace("\t", "").replace("|", " | ").strip()  
            else:
                Group_type = ""  

            if Genre_Text:
                Genre_Text = Genre_Text.text.replace(",", "-").strip()  
            else:
                Genre_Text = ""  

            
            if Company:
                Company = Company.text.strip()  
            else:
                Company = ""  

            print("INSERT INTO idol_Group (Group_Code, Group_Name, Group_ImgUrl, debut_Year, debut_song, Year_Active, Group_type, Genre_Text, Company) VALUES ({0},'{1}','{2}','{3}','{4}','{5}','{6}','{7}','{8}')".format(i,G_Name_text,img_url,debut_year_text, debut_song,Year_Active,Group_type, Genre_Text,Company))  # 리스트 대신 문자열 출력
# 드라이버 종료
driver.quit()


INSERT INTO idol_Group (Group_Code, Group_Name, Group_ImgUrl, debut_Year, debut_song, Year_Active, Group_type, Genre_Text, Company) VALUES (995173,'제니 (JENNIE)','https://cdnimg.melon.co.kr/cm2/artistcrop/images/009/95/173/995173_20241010112324_500.jpg?f086e63a0f6b973fea79264297f49b56/melon/resize/416/quality/80/optimize','2010, 2020 년대','None','솔로 | 여성','국외드라마, 댄스, POP, 랩/힙합, R&B/Soul','(주)YG엔터테인먼트','')
INSERT INTO idol_Group (Group_Code, Group_Name, Group_ImgUrl, debut_Year, debut_song, Year_Active, Group_type, Genre_Text, Company) VALUES (712452,'AKMU (악뮤)','https://cdnimg.melon.co.kr/cm2/artistcrop/images/007/12/452/712452_20240603132217_500.jpg?5895e8fcf778b78ce3385759f95fe848/melon/resize/416/quality/80/optimize','2014.04.07','None','2010, 2020 년대','그룹 | 혼성','댄스- 포크/블루스- 발라드- 록/메탈- 국내드라마- R&B/Soul- 일렉트로니카- 랩/힙합- 재즈- 애시드/퓨전/팝','YG Family')
INSERT INTO idol_Group (Group_Code, Group_Name, Group_ImgUrl, debut_Year, debut_song, Year_Active, Group_type, Genre_Text, Company) VALUES (2399

In [13]:
## DB에 있는 IMG와 멜론에 등록된 앨범IMG의 URL를 비교함

import mysql.connector

# 데이터베이스 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='music',
    database='MusicDatabase'
)

cursor = conn.cursor()


# 최신일자 Top10에 대한 싱어코드를 추출하여 DB에 등록된 앨범이미지의 URL과 멜론 사이트의 앨범이미지 URL을 비교
query = """
    select Singer_Code From Music where 1=1 and ranking < 11 and day = (select Max(day) from Music) and Singer_Code is not null group by Singer_Code;
"""
cursor.execute(query)

# 결과 출력
Group_Code_List = cursor.fetchall()
for (Group_Code) in Group_Code_List:
    print(Group_Code)

# 커서와 연결 닫기
cursor.close()
conn.close()



(995173,)
(712452,)
(2399724,)
(2899555,)
(894864,)
(3610529,)
(2019210,)
(3752136,)


In [16]:
Group_Code_List

# 튜플의 첫 번째 요소만 추출
group_Codes = [code[0] for code in Group_Code_List]

print(group_Codes)


[995173, 712452, 2399724, 2899555, 894864, 3610529, 2019210, 3752136]


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import mysql.connector

# ChromeDriver 경로 설정
driver_path = 'C:\\P_Project\\chromedriver-win64\\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# MySQL 연결 설정
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='music',
    database='MusicDatabase'
)
cursor = db.cursor()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for i in group_Codes:
    # 1. 크롤링: 그룹 코드에 대해 이미지 URL을 가져옴
    URL = f"https://www.melon.com/artist/timeline.htm?artistId={i}"
    driver.get(URL)
    time.sleep(1)  # 페이지 로딩 대기

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 이미지 URL 추출
    group_img_url = soup.select_one('span#artistImgArea img')['src'] if soup.select_one('span#artistImgArea img') else ''

    # 2. 위에서 싱어코드를 추출하여 DB에 등록된 앨범이미지의 URL과 멜론 사이트의 앨범이미지 URL을 비교
    query = f"SELECT A.Group_ImgUrl FROM idol_Group A, Music B WHERE A.Group_Code = {i} and A.Group_Name = B.Singer and B.Ranking < 11 Group by A.Group_ImgUrl"
    cursor.execute(query)
    db_img_url = cursor.fetchone()

    # 3. 비교 및 업데이트
    if db_img_url:
        db_img_url = db_img_url[0]  # fetchone은 튜플로 반환되므로 첫 번째 값 가져오기
        if group_img_url != db_img_url:  # 이미지 URL이 다를 경우
            update_query = f"UPDATE idol_Group SET Group_ImgUrl = %s WHERE Group_Code = %s"
            cursor.execute(update_query, (group_img_url, i))
            db.commit()  # 업데이트 적용
            print(f"Group_Code {i}의 이미지 URL을 업데이트했습니다.")
            print(f"SQL Query: UPDATE idol_Group SET Group_ImgUrl = '{group_img_url}' WHERE Group_Code = {i};")
        else:
            print(f"Group_Code {i}의 이미지 URL이 동일합니다.")
    else:
        print(f"Group_Code {i}에 대한 정보가 없습니다.")

# 드라이버 종료 및 DB 연결 해제
driver.quit()
cursor.close()
db.close()

Group_Code 995173의 이미지 URL이 동일합니다.
Group_Code 712452의 이미지 URL이 동일합니다.
Group_Code 2399724의 이미지 URL이 동일합니다.
Group_Code 2899555의 이미지 URL이 동일합니다.
Group_Code 894864의 이미지 URL이 동일합니다.
Group_Code 3610529의 이미지 URL이 동일합니다.
Group_Code 2019210의 이미지 URL이 동일합니다.
Group_Code 3752136의 이미지 URL이 동일합니다.
